In [16]:
import os
import sys
os.chdir("/Users/zyxu/Documents/py/kris")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from func.prepro import SingleCellDataset, SingleCellDataset_test


In [17]:
DATA_DIR = "open-problems-multimodal"
SUBMIT_DIR = "submit"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

In [18]:
# test_multi = SingleCellDataset_test(FP_MULTIOME_TEST_INPUTS,testresponse =20)
test_multi = SingleCellDataset(FP_MULTIOME_TEST_INPUTS)
testloader_multi = DataLoader(test_multi, batch_size=64)
# train_multi = SingleCellDataset_test(FP_MULTIOME_TRAIN_INPUTS,FP_MULTIOME_TRAIN_TARGETS,testresponse =20)
train_multi = SingleCellDataset(FP_MULTIOME_TRAIN_INPUTS,FP_MULTIOME_TRAIN_TARGETS)
trainloader_multi = DataLoader(train_multi, batch_size=64)

In [19]:
class myModel(nn.Module):
    def __init__(self) -> None:
        super(myModel, self).__init__()

        self.LR = nn.Linear(1000,100, bias=True)

    def forward(self, input):
        output = self.LR(input)
        return output

model = myModel()
# We want to investigate whether A = M^T M is closed to identity
print(model.LR)

input = torch.randn(1000)

Linear(in_features=1000, out_features=100, bias=True)


In [20]:
model(input)

tensor([ 0.8570,  0.0342, -0.4760,  0.6868,  0.1540,  0.0523, -0.0306, -0.0979,
        -0.1501,  0.1945,  0.9579,  0.2288, -0.5478,  0.6643, -0.0269,  0.4266,
         0.8672, -0.6577,  0.6365,  1.3117,  0.6636, -0.2918, -0.3952,  0.4048,
         0.0255,  0.3892, -0.0582,  1.0805,  0.2535, -0.7339,  0.6404, -0.2711,
        -0.4150,  0.8145, -0.2030, -0.1711,  0.4961, -0.2938, -0.3027,  0.2293,
        -0.6180,  0.5767, -0.1383,  0.6431, -0.4321, -0.1949, -0.7534,  0.0339,
         0.7144, -0.7036,  0.4395, -1.1806, -0.0400,  0.8111,  0.6929,  0.4079,
        -0.3526, -0.6184, -0.3044,  0.2565, -0.2569, -0.1088, -0.1435,  0.4048,
         0.9161, -0.0278, -0.7103, -0.2775, -0.4009,  0.3164,  0.2217,  0.3494,
        -0.0729,  0.2960,  0.4181,  0.3935, -0.2383,  0.1385,  0.4245, -0.1076,
        -0.3278,  0.8333, -0.1533, -0.2451, -0.0575, -0.1631, -0.8299,  0.7458,
         0.0458, -0.2423, -1.0523, -0.2650, -0.0598, -0.0394, -0.6913, -0.2668,
        -0.2695, -0.1473, -0.3432, -0.00

In [21]:
BATCH_SIZE = 1024
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 0
MOMENTUM = 0

In [22]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_id, (cells, features, targets) in enumerate(train_loader):
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward
        outputs = model(features)
        loss = F.mse_loss(targets,outputs)
        loss.backward()

        # optimize
        optimizer.step()

        # LOGGING
        if not batch_id % 50:
               print("Epoch: {}  |  Batch: {}/{}  |  Cost: {}".format(epoch+1, batch_id, len(train_loader), loss))


In [23]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)


In [24]:
# training loop
train_loss_total = []
for epoch in range(20):
    train_loss = train(model, trainloader_multi, optimizer, epoch)
    train_loss_total.append(train_loss)
    # LOGGING
    if epoch % 5000 == 0:
        print("Epoch: {}/{}  === Train Cost: {}".format(epoch + 1, 20000, train_loss))


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x228942 and 1000x100)